In [ ]:
import pandas as pd
from pandas_datareader import data
import yfinance as yf
yf.pdr_override()




In [ ]:
def get_macd(price, slow, fast, smooth):
    exp1 = price.ewm(span = fast, adjust = False).mean()
    exp2 = price.ewm(span = slow, adjust = False).mean()
    macd = pd.DataFrame(exp1 - exp2).rename(columns = {'Close':'macd'})
    signal = pd.DataFrame(macd.ewm(span = smooth, adjust = False).mean()).rename(columns = {'macd':'signal'})
    hist = pd.DataFrame(macd['macd'] - signal['signal']).rename(columns = {0:'hist'})
    frames =  [macd, signal, hist]
    df = pd.concat(frames, join = 'inner', axis = 1)
    return df

def macd_diff(df,i):
    return df.loc[i,'macd']-df.loc[i,'signal']
    
def get_almost_macd_signal(df,hold,i):
    if (i<2):
        return False
    if (hold):
        if (macd_diff(df,i)<macd_diff(df,i-1) and macd_diff(df,i-1)<macd_diff(df,i-2) and macd_diff(df,i)>0 and macd_diff(df,i)<0.01):
            return True
        return False
    else:
        if (macd_diff(df,i)>macd_diff(df,i-1) and macd_diff(df,i-1)>macd_diff(df,i-2) and macd_diff(df,i)<0 and macd_diff(df,i)>-0.01):
            return True
        return False

def get_macd_signal(df,hold,i):
    if (hold):
        if ((df.iloc[i])['macd']<(df.iloc[i])['signal']):
            return True
        return False
    else:
        if ((df.iloc[i])['macd']>(df.iloc[i])['signal']):
            return True
        return False

def get_macd_signal_new(df,hold,i):
    if (i<1):
        return False
    
    if (hold):
        if (((df.iloc[i-1])['macd']>(df.iloc[i-1])['signal']) and ((df.iloc[i])['macd']<(df.iloc[i])['signal'])):
            return True
        return False
    else:
        if (((df.iloc[i-1])['macd']<(df.iloc[i-1])['signal']) and ((df.iloc[i])['macd']>(df.iloc[i])['signal'])):
            return True
        return False    
     
def naive_backtest(df,get_signal):
    pv=1000
    hold=False
    start_price=-float("inf")
    
    total_trade=0
    win_count=0
    
    #count=0
    for i in range(len(df)):
        if (hold):
            if (get_signal(df,hold,i)):
                end_price=df.iloc[i]['Adj Close']
                pv*=end_price/start_price
                hold=False
                
                total_trade+=1
                if (end_price>start_price):
                    win_count+=1
        else:
            if (get_signal(df,hold,i)):
                #count+=1
                start_price=df.iloc[i]['Adj Close']
                hold=True
    #print(count) 
    return (pv,total_trade,win_count)


def backtest_one_day(df,get_signal):
    pv=1000

    total_trade=0
    win_count=0
    
    #count=0
    for i in range(len(df)-1):
        if (get_signal(df,False,i)):
            #count+=1
            start_price=df.iloc[i]['Adj Close']
            end_price=df.iloc[i+1]['Adj Close']
            pv*=end_price/start_price

            total_trade+=1
            if (end_price>start_price):
                win_count+=1
    #print(count)
    return (pv,total_trade,win_count)

def backtest_one_day_short_possible(df,get_signal):
    pv=1000

    total_trade=0
    win_count=0
    
    for i in range(len(df)-1):
        if (get_signal(df,False,i)):
            start_price=df.iloc[i]['Adj Close']
            end_price=df.iloc[i+1]['Adj Close']
            pv*=end_price/start_price
            
            total_trade+=1
            if (end_price>start_price):
                win_count+=1
        elif (get_signal(df,True,i)):
            start_price=df.iloc[i]['Adj Close']
            end_price=df.iloc[i+1]['Adj Close']
            pv*=start_price/end_price
            
            total_trade+=1
            if (end_price<start_price):
                win_count+=1
        
    return (pv,total_trade,win_count)

In [ ]:
#find how many times there will be earning when an MACD golden crossover appears.
temp=[]

counts={}
for span in range(3,10):
    total_count=0
    long_count=0
    for t in ['601318.ss','603938.ss','002229.sz','603722.ss','600519.ss','300065.sz','000002.sz','000010.sz','000035.sz','301487.sz','300570.sz'\
             '688359.ss','301203.sz','000564.sz','002362.sz','688333.ss','002230.sz','002415.sz','002262.sz','001979.sz','601099.ss'\
             ]:
        df=data.get_data_yahoo(tickers=t,start='2010-01-01',end='2023-08-01')
        df_macd = get_macd(df['Close'], 26, 12, span)
        df_enlarged=pd.concat([df,df_macd],axis=1)[1:].reset_index()
        for i in range(1,len(df_enlarged)):
            if (macd_diff(df_enlarged,i)>0 and macd_diff(df_enlarged,i-1)<0):
                total_count+=1
                if(df_enlarged.loc[i,'Close']>df_enlarged.loc[i-1,'Close']):
                    long_count+=1
    counts[span]=(long_count,total_count)

temp=[]
for span in range(3,10):
    temp.append(list(counts[span]))

res=[]
for _ in range(len(temp)):
    tmp=[0]*len(temp[0])
    res.append(tmp)
df_plot=pd.DataFrame(res)

for i in range(len(temp)):
    for j in range(len(temp[0])):
        df_plot.loc[i,j]=temp[i][j]
        
df_plot.to_excel("long_on_the_day_of_crossover.xlsx")

In [ ]:
#对应“MACD金死叉对单调2”表格
temp=[]
for t in ['601318.ss','603938.ss','002229.sz','603722.ss','600519.ss','300065.sz','000002.sz','000010.sz','000035.sz','301487.sz','300570.sz'\
         '688359.ss','301203.sz','000564.sz','002362.sz','688333.ss','002230.sz','002415.sz','002262.sz','001979.sz','601099.ss']:
    df=data.get_data_yahoo(tickers=t,start='2010-01-01',end='2023-08-01')
    df_macd = get_macd(df['Close'], 26, 12, 9)
    df_enlarged=pd.concat([df,df_macd],axis=1)[1:].reset_index()
    
    temp.append([])
    a=naive_backtest(df_enlarged,get_macd_signal)
    print(a,end="")
    temp[-1].extend(a)
    
    a=naive_backtest(df_enlarged,get_almost_macd_signal)
    print(a)
    temp[-1].extend(a)

In [ ]:
temp=[]
for t in ['601318.ss','603938.ss','002229.sz','603722.ss','600519.ss','300065.sz','000002.sz','000010.sz','000035.sz',\
         '688359.ss','301203.sz','000564.sz','002362.sz','688333.ss','002230.sz','002415.sz','002262.sz','001979.sz','601099.ss']:
    df=data.get_data_yahoo(tickers=t,start='2010-01-01',end='2023-08-01')
    df_macd = get_macd(df['Close'], 26, 12, 9)
    df_enlarged=pd.concat([df,df_macd],axis=1)[1:].reset_index()
    
    temp.append([])
    a=backtest_one_day(df_enlarged,get_macd_signal)
    print(a,end="")
    temp[-1].extend(a)
    
    a=backtest_one_day(df_enlarged,get_almost_macd_signal)
    print(a)
    temp[-1].extend(a)

In [ ]:
temp=[]
for t in ['601318.ss','603938.ss','002229.sz','603722.ss','600519.ss','300065.sz','000002.sz','000010.sz','000035.sz',\
         '688359.ss','301203.sz','000564.sz','002362.sz','688333.ss','002230.sz','002415.sz','002262.sz','001979.sz','601099.ss']:
    df=data.get_data_yahoo(tickers=t,start='2010-01-01',end='2023-08-01')
    df_macd = get_macd(df['Close'], 26, 12, 9)
    df_enlarged=pd.concat([df,df_macd],axis=1)[1:].reset_index()
    
    temp.append([])
    #a=backtest_one_day_short_possible(df_enlarged,get_macd_signal_new)
    #print(a,end="")
    #temp[-1].extend(a)
    
    a=backtest_one_day_short_possible(df_enlarged,get_almost_macd_signal)
    print(a)
    temp[-1].extend(a)

In [ ]:
temp=[]
for span in range(3,10):
    temp.append(list(counts[span]))

res=[]
for _ in range(len(temp)):
    tmp=[0]*len(temp[0])
    res.append(tmp)
df_plot=pd.DataFrame(res)

for i in range(len(temp)):
    for j in range(len(temp[0])):
        df_plot.loc[i,j]=temp[i][j]
        
df_plot.to_excel("long_on_the_day_of_crossover.xlsx")